In [1]:
import pandas as pd

In [2]:
hotel_df = pd.read_csv('hotel.csv')

display(hotel_df.head())
print(hotel_df.info())

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,Resort Hotel,0,342,1,July,27,1,0,0,2,...,0,0,0,C,C,3,Transient,0.0,0,0
1,Resort Hotel,0,7,1,July,27,1,0,1,1,...,0,0,0,A,C,0,Transient,75.0,0,0
2,Resort Hotel,0,13,1,July,27,1,0,1,1,...,0,0,0,A,A,0,Transient,75.0,0,0
3,Resort Hotel,0,14,1,July,27,1,0,2,2,...,0,0,0,A,A,0,Transient,98.0,0,1
4,Resort Hotel,0,0,1,July,27,1,0,2,2,...,0,0,0,C,C,0,Transient,107.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85913 entries, 0 to 85912
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           85913 non-null  object 
 1   is_canceled                     85913 non-null  int64  
 2   lead_time                       85913 non-null  int64  
 3   arrival_date_year               85913 non-null  int64  
 4   arrival_date_month              85913 non-null  object 
 5   arrival_date_week_number        85913 non-null  int64  
 6   arrival_date_day_of_month       85913 non-null  int64  
 7   stays_in_weekend_nights         85913 non-null  int64  
 8   stays_in_week_nights            85913 non-null  int64  
 9   adults                          85913 non-null  int64  
 10  children                        85913 non-null  float64
 11  meal                            85913 non-null  object 
 12  market_segment                  

In [3]:
## Create X (features) and y(response)
X_features = hotel_df.drop('is_canceled', axis=1)
display(X_features.head())
y_target = hotel_df.is_canceled
display(y_target.head())

# checking frequncy of target classes
y_target.value_counts()

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,Resort Hotel,342,1,July,27,1,0,0,2,0.0,...,0,0,0,C,C,3,Transient,0.0,0,0
1,Resort Hotel,7,1,July,27,1,0,1,1,0.0,...,0,0,0,A,C,0,Transient,75.0,0,0
2,Resort Hotel,13,1,July,27,1,0,1,1,0.0,...,0,0,0,A,A,0,Transient,75.0,0,0
3,Resort Hotel,14,1,July,27,1,0,2,2,0.0,...,0,0,0,A,A,0,Transient,98.0,0,1
4,Resort Hotel,0,1,July,27,1,0,2,2,0.0,...,0,0,0,C,C,0,Transient,107.0,0,0


0    0
1    0
2    0
3    0
4    0
Name: is_canceled, dtype: int64

0    62401
1    23512
Name: is_canceled, dtype: int64

## List of numerical and categorical columns:

In [4]:
num_int_type = X_features.select_dtypes(include='int64').columns.tolist()
num_float_type = X_features.select_dtypes(include='float64').columns.tolist()

num_type = num_int_type + num_float_type
print("\n\nNumerical Types columns name: \n", num_type)

cat_object_type = X_features.select_dtypes(include='object').columns.tolist()
print("\n\nCategorical Types columns name: \n", cat_object_type)



Numerical Types columns name: 
 ['lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'required_car_parking_spaces', 'total_of_special_requests', 'children', 'adr']


Categorical Types columns name: 
 ['hotel', 'arrival_date_month', 'meal', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'customer_type']


In [5]:
## Encoding categoirical varriables to "0" and "1" or numeric
## scaling numerical variables with mean "0" and standard devaition "1"

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

from sklearn.compose import make_column_transformer
column_trans = make_column_transformer((OneHotEncoder(sparse=False, handle_unknown='error'), cat_object_type), 
                                     (StandardScaler(), num_type), remainder = 'passthrough')

# check
check = column_trans.fit_transform(X_features)
display(check)

# # convert numpy array into pandas dataframe
# df = pd.DataFrame(data=check)
# display(df)

array([[ 0.        ,  1.        ,  0.        , ..., -0.84127719,
        -0.30576108, -1.93632564],
       [ 0.        ,  1.        ,  0.        , ..., -0.84127719,
        -0.30576108, -0.57446615],
       [ 0.        ,  1.        ,  0.        , ..., -0.84127719,
        -0.30576108, -0.57446615],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  3.96874843,
        -0.30576108,  0.92739249],
       [ 1.        ,  0.        ,  0.        , ..., -0.84127719,
        -0.30576108, -0.04061723],
       [ 1.        ,  0.        ,  0.        , ...,  1.56373562,
        -0.30576108,  0.80918309]])

In [6]:
## Splitiing hotel datasets into training and testing datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size = 0.3, random_state=0)

In [7]:
from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import LogisticRegression

# logreg_pipe = make_pipeline(column_trans, LogisticRegression(solver='lbfgs', max_iter=10000))

In [8]:

# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold

# # split1 = KFold(n_splits=5, shuffle=False, random_state=None)
# split2 = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# cross_val_score(logreg_pipe, X_train, y_train, cv=split2, scoring='accuracy').mean()

In [9]:
## Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg_pipe = make_pipeline(column_trans, LogisticRegression(solver='lbfgs', max_iter=10000))

logreg_pipe.fit(X_train, y_train)
y_pred = logreg_pipe.predict(X_test)


from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

0.7865678590827966
[[17378  1387]
 [ 4114  2895]]
              precision    recall  f1-score   support

           0       0.81      0.93      0.86     18765
           1       0.68      0.41      0.51      7009

    accuracy                           0.79     25774
   macro avg       0.74      0.67      0.69     25774
weighted avg       0.77      0.79      0.77     25774



In [10]:
## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt_pipe = make_pipeline(column_trans, DecisionTreeClassifier())

dt_pipe.fit(X_train, y_train)
y_pred = dt_pipe.predict(X_test)


from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

0.7538992783425157
[[15515  3250]
 [ 3093  3916]]
              precision    recall  f1-score   support

           0       0.83      0.83      0.83     18765
           1       0.55      0.56      0.55      7009

    accuracy                           0.75     25774
   macro avg       0.69      0.69      0.69     25774
weighted avg       0.76      0.75      0.75     25774



In [11]:
## Naive Bayes
# GaussianNB implements the Gaussian Naive Bayes algorithm for classification.
from sklearn.naive_bayes import GaussianNB
gnb_pipe = make_pipeline(column_trans, GaussianNB())

gnb_pipe.fit(X_train, y_train)
y_pred = gnb_pipe.predict(X_test)

from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

0.730581205866377
[[18613   152]
 [ 6792   217]]
              precision    recall  f1-score   support

           0       0.73      0.99      0.84     18765
           1       0.59      0.03      0.06      7009

    accuracy                           0.73     25774
   macro avg       0.66      0.51      0.45     25774
weighted avg       0.69      0.73      0.63     25774



In [12]:
# ## Support Vector Machines
# from sklearn import svm
# svm_SVC_pipe = make_pipeline(column_trans, svm.SVC(kernel='linear')) # Linear Kernel for now

# svm_SVC_pipe.fit(X_train, y_train)
# y_pred = svm_SVC_pipe.predict(X_test)


# from sklearn import metrics
# print(metrics.accuracy_score(y_test, y_pred))
# print(metrics.confusion_matrix(y_test, y_pred))
# print(metrics.classification_report(y_test, y_pred))

In [13]:
## Neural Networks
from sklearn.neural_network import MLPClassifier
MLP_pipe = make_pipeline(column_trans, MLPClassifier(hidden_layer_sizes=(13,13,13), max_iter=500))

MLP_pipe.fit(X_train, y_train)
y_pred = MLP_pipe.predict(X_test)


from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

0.8090323581904244
[[17139  1626]
 [ 3296  3713]]
              precision    recall  f1-score   support

           0       0.84      0.91      0.87     18765
           1       0.70      0.53      0.60      7009

    accuracy                           0.81     25774
   macro avg       0.77      0.72      0.74     25774
weighted avg       0.80      0.81      0.80     25774

